In [149]:
!pip3 install torchtext==0.4.0

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# Train MLM

In [150]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=25000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class MaskedLanguageModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.1):
        super().__init__()
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = nn.TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [151]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [152]:
import torchtext
from torchtext.data.utils import get_tokenizer

TEXT = torchtext.data.Field(tokenize='spacy', init_token='<sos>', eos_token='<eos>', lower=True)
LABEL = torchtext.data.LabelField(dtype = torch.float)
train_data, test_data = torchtext.datasets.TREC.splits(TEXT, LABEL)
train_data, valid_data = train_data.split()
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = 64,
    sort_within_batch = True,
    device = device)

In [153]:
bptt = 64
# def get_batch(source, i):
#     seq_len = min(bptt, len(source) - 1 - i)
#     data = source[i:i+seq_len]
#     target = source[i+1:i+1+seq_len].reshape(-1)
#     return data, target

In [154]:
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 64 # embedding dimension
nhid = 64 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
model = MaskedLanguageModel(ntokens, emsize, nhead, nhid, nlayers, dropout)
model = model.to(device)

In [155]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in zip(train_iterator, range(len(train_iterator))):
        data = batch.text
        # data, targets = get_batch(batch.text, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0))
            src_mask = src_mask.to(device)
        output = model(data, src_mask)
        # print('output.view(-1, ntokens):', output.shape)
        # print('data.reshape(-1)', data.reshape(-1).shape)
        loss = criterion(output.view(data.reshape(-1).shape[0], -1), data.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 10
        if i % log_interval == 0 and i > 0:
            # cur_loss = total_loss / len(train_iterator)
            cur_loss = loss.item()
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  ' ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, i, len(train_data) // bptt,
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_iterator):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for batch, i in zip(data_iterator, range(len(data_iterator))):
            data = batch.text
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0))
                src_mask = src_mask.to(device)
            output = eval_model(data, src_mask)
            # print(output.shape)
            # print(data.shape)
            # output_flat = output.view(-1, ntokens)
            loss_ = len(data) * criterion(output.view(data.reshape(-1).shape[0], -1), data.reshape(-1)).item()
            total_loss += loss_
    return total_loss / len(data_iterator)

In [156]:
best_val_loss = float("inf")
losses = []
epochs = 50 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, valid_iterator)
    losses.append(val_loss)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
    elif epoch >=3:
        if val_loss >= losses[-1] and val_loss >= losses[-2]:
            print('early stopping')
            torch.save(best_model.state_dict(), 'MLM_wikitext.pt')
            break 

| epoch   1 |    10/   59 batches |  ms/batch 24.78 | loss  7.89 | ppl  2669.74
| epoch   1 |    20/   59 batches |  ms/batch 18.98 | loss  7.07 | ppl  1180.75
| epoch   1 |    30/   59 batches |  ms/batch 19.29 | loss  6.21 | ppl   498.43
| epoch   1 |    40/   59 batches |  ms/batch 18.63 | loss  5.72 | ppl   305.84
| epoch   1 |    50/   59 batches |  ms/batch 19.22 | loss  4.66 | ppl   105.77
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  1.32s | valid loss 62.04 | valid ppl 879311135729863882165977088.00
-----------------------------------------------------------------------------------------
| epoch   2 |    10/   59 batches |  ms/batch 21.17 | loss  4.97 | ppl   144.02
| epoch   2 |    20/   59 batches |  ms/batch 18.66 | loss  2.82 | ppl    16.73
| epoch   2 |    30/   59 batches |  ms/batch 19.05 | loss  3.10 | ppl    22.30
| epoch   2 |    40/   59 batches |  ms/batch 18.88 | loss  3.29 | ppl    26.83
| ep

In [157]:
test_loss = evaluate(best_model, test_iterator)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  8.92 | test ppl  7490.68


------

In [158]:
# #load pretrained model
# model.load_state_dict(torch.load('MLM_wikitext.pt'))

# Train Text GRU.

In [159]:
class TextGRU(nn.Module):
    '''
    Recurrent Neural Network based on Gated Rectified Unit for text classification [1].
    Credits for implementation: [2]
    Inputs:
        vocab_size - size of vocabulary of dataset used
        emb_dim - embedding space dimension
        hidden_dim - dimension of hidden state of RNN
        output_dim - dimension of the output (number of classes in our case)
        dropout - drop out probability
        pad_idx - padding index for sequences
    [1]: Empirical evaluation of gated recurrent neural networks on sequence modeling, 2014.
    [2]: https://github.com/etomoscow/DL-in-NLP/blob/master/hw3/task3_sentiment_rnn.ipynb
    '''

    def __init__(self, vocab_size, emb_dim, hidden_dim, out_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(emb_dim, hidden_dim, num_layers=1, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, hidden = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        return self.fc(hidden)

    def forward_on_embeddings(self, embed, text_lengths):
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embed, text_lengths)
        packed_output, hidden = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        return self.fc(hidden) 

In [160]:
#load the classifier

classifier = TextGRU(vocab_size=len(TEXT.vocab), emb_dim=100, hidden_dim=128, out_dim=6, dropout=0.1, pad_idx=TEXT.vocab.stoi[TEXT.pad_token])
# classifier.load_state_dict(torch.load('textrnn_trec.pt'))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [161]:
criterion_ = nn.CrossEntropyLoss()
criterion_ = criterion_.to(device)
classifier = classifier.to(device)
optimizer_ = torch.optim.Adam(classifier.parameters(), lr=0.001)

In [162]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim=1, keepdim=True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    correct = correct.detach().to('cpu')
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [163]:
def get_text_length(batch):
    global device
    global TEXT
    result = []
    for i in range(batch.shape[1]):
        result.append((sum(batch[:, i] != TEXT.vocab.stoi[TEXT.pad_token]).item()))
    return torch.tensor(result, dtype=int, device='cpu')

In [164]:
def train_gru(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text = batch.text
        text_lengths = get_text_length(text)
        predictions = model(text, text_lengths)
        loss = criterion(predictions, batch.label.long())
        acc = categorical_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [165]:
def eval_gru(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            text_lengths = get_text_length(text)
            predictions = model(text, text_lengths)
            loss = criterion(predictions, batch.label.long())
            acc = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [166]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [167]:
N_EPOCHS = 50
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train_gru(classifier, train_iterator, optimizer_, criterion_)
    valid_loss, valid_acc = eval_gru(classifier, valid_iterator, criterion_)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'textrnn_trec.pt')
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.316 | Train Acc: 48.76%
	 Val. Loss: 1.026 |  Val. Acc: 59.77%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.862 | Train Acc: 67.43%
	 Val. Loss: 0.792 |  Val. Acc: 69.98%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.647 | Train Acc: 76.08%
	 Val. Loss: 0.745 |  Val. Acc: 72.43%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.484 | Train Acc: 82.67%
	 Val. Loss: 0.717 |  Val. Acc: 75.10%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.348 | Train Acc: 87.52%
	 Val. Loss: 0.757 |  Val. Acc: 75.01%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 0.244 | Train Acc: 92.37%
	 Val. Loss: 0.773 |  Val. Acc: 77.19%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 0.158 | Train Acc: 95.21%
	 Val. Loss: 0.844 |  Val. Acc: 75.76%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 0.100 | Train Acc: 97.21%
	 Val. Loss: 0.901 |  Val. Acc: 76.60%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 0.066 | Train Acc: 98.29%
	 Val. Loss: 0.911 |  Val. Acc: 77.01%
Epoch: 10 | Epoch Time: 0m 1

# Prepare baseline attack - SamplingFool

SamplingFool 

`output` is a 3D-tensor of shape `[sequnce_len, batch_size, vocab_size]`

In [177]:
model.eval() # Turn on the evaluation mode
classifier.eval() #turn on evaluation mode for classifier
with torch.no_grad():
    for batch in test_iterator:
        fullbatch = batch
        data = batch.text
        src_mask = model.generate_square_subsequent_mask(data.size(0))
        src_mask = src_mask.to(device)
        output = model(data, src_mask)
        for tau in torch.arange(1e-2, 1e5, 20):
            gumbel_output = torch.nn.functional.gumbel_softmax(output, tau=tau, hard=True, eps=1e-10, dim=2)
            try:
                attack_output = classifier(torch.argmax(gumbel_output, dim=2), get_text_length(torch.argmax(gumbel_output, dim=2)))
            except RuntimeError:
                continue
            original_output = classifier(data, get_text_length(data))
            if categorical_accuracy(attack_output, fullbatch.label).item() + 0.1 < categorical_accuracy(original_output, fullbatch.label).item():
                print('Attack was successfull, accuracy drop: {:.3f}%, tau value:{}'.format((categorical_accuracy(attack_output, fullbatch.label).item()-categorical_accuracy(original_output, fullbatch.label).item())*100, tau))

Attack was successfull, accuracy drop: -10.938%, tau value:4360.009765625
Attack was successfull, accuracy drop: -10.938%, tau value:4460.009765625
Attack was successfull, accuracy drop: -12.500%, tau value:22540.009765625
Attack was successfull, accuracy drop: -10.938%, tau value:35780.01171875
Attack was successfull, accuracy drop: -10.938%, tau value:40080.01171875
Attack was successfull, accuracy drop: -10.938%, tau value:49900.01171875
Attack was successfull, accuracy drop: -10.938%, tau value:51400.01171875
Attack was successfull, accuracy drop: -12.500%, tau value:57320.01171875
Attack was successfull, accuracy drop: -12.500%, tau value:62620.01171875
Attack was successfull, accuracy drop: -10.938%, tau value:68120.0078125
Attack was successfull, accuracy drop: -10.938%, tau value:70640.0078125
Attack was successfull, accuracy drop: -10.938%, tau value:74780.0078125
Attack was successfull, accuracy drop: -10.938%, tau value:88680.0078125
Attack was successfull, accuracy drop: -1

KeyboardInterrupt: ignored